载入配置

In [1]:
from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, evaluate_model, interpret_model
import pandas as pd
import sys, os

sys.path.append('../')

from src import proj_dir

Use environmental variables in default.yml


载入测试数据

In [2]:
numeric_cols = [
    'AGE', 'ISS', 'CAPRINI_SCORE', 'T', 'P', 'R', 'MBP', 'SHOCK_INDEX', 'HEIGHT', 'WEIGHT',
    'BMI', 'RBC', 'HGB', 'PLT', 'WBC', 'ALB', 'CRE', 'UA', 'AST', 'ALT', 'GLU', 'TG', 'CHO',
    'CA', 'MG', 'LDL', 'NA', 'K', 'CL', 'GFR', 'PT', 'FIB', 'DD', 'CK', 'INR'
]

data = pd.read_csv(os.path.join(proj_dir, 'data/local_test/patient_data.csv'))
data.drop('INPATIENT_ID', axis = 1, inplace = True)

for col in data.columns:
    if col not in numeric_cols:
        data[col] = data[col].astype(int)

创建分类模型

In [3]:
setup(data, target = 'VTE', numeric_features = numeric_cols, verbose = False)

INFO:logs:create_model_container 0
INFO:logs:master_model_container 0
INFO:logs:display_container 0
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      ml_usecase='classification',
                                      numerical_features=['AGE', 'ISS',
                                                          'CAPRINI_SCORE', 'T',
                                                          'P', 'R', 'MBP',
                                                          'SHOCK_INDEX',
                                                          'HEIGHT', 'WEIGHT',
                                                          'BMI', 'RBC', 'HGB',
                                                          'PLT', 'WBC', 'ALB',
                                                          'CRE', 'UA', 'AST',
                   

(      AGE   ISS  CAPRINI_SCORE     T      P     R         MBP  SHOCK_INDEX  \
 0    33.0  17.0            4.0  36.5   78.0  20.0   76.666667     0.780000   
 1    19.0  29.0           10.0  36.5   67.0  20.0   79.666667     0.592920   
 2    46.0  34.0            8.0  36.6   98.0  20.0   82.666667     0.844828   
 3    51.0  29.0           11.0  36.5  110.0  17.0   79.000000     1.182796   
 4    56.0  32.0           11.0  36.6   52.0  18.0  106.333333     0.379562   
 ..    ...   ...            ...   ...    ...   ...         ...          ...   
 893  46.0   4.0            3.0  36.5   96.0  20.0  109.333333     0.695652   
 894  71.0   4.0            5.0  36.3   75.0  20.0   98.333333     0.572519   
 895  54.0   4.0            4.0  36.5   83.0  20.0   84.333333     0.747748   
 896  27.0   5.0            3.0  36.6   88.0  20.0   84.666667     0.800000   
 897  38.0   4.0            2.0  36.3   96.0  20.0  104.666667     0.738462   
 
      HEIGHT  WEIGHT  ...  PD_0  PD_1  STROKE_0  S

模型比较

In [4]:
# 自动打印各模型各指标对比表.
# 创建模型和训练调参.
# ID          Name
# --------    ----------
# 'lr'        Logistic Regression
# 'knn'       K Nearest Neighbour
# 'nb'        Naive Bayes
# 'dt'        Decision Tree Classifier
# 'svm'       SVM - Linear Kernel
# 'rbfsvm'    SVM - Radial Kernel
# 'gpc'       Gaussian Process Classifier
# 'mlp'       Multi Level Perceptron
# 'ridge'     Ridge Classifier
# 'rf'        Random Forest Classifier
# 'qda'       Quadratic Discriminant Analysis
# 'ada'       Ada Boost Classifier
# 'gbc'       Gradient Boosting Classifier
# 'lda'       Linear Discriminant Analysis
# 'et'        Extra Trees Classifier
# 'xgboost'   Extreme Gradient Boosting
# 'lightgbm'  Light Gradient Boosting
# 'catboost'  CatBoost Classifier
model_store_final = compare_models(
    whitelist = ['lr', 'rf', 'ridge', 'xgboost'],
    sort = 'F1',
    verbose = True
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extreme Gradient Boosting,0.8106,0.7465,0.3242,0.6056,0.4119,0.3152,0.3410,0.5174
1,Ridge Classifier,0.7962,0.0000,0.2681,0.5300,0.3496,0.2483,0.2693,0.0339
2,Logistic Regression,0.7899,0.7485,0.2022,0.5686,0.2834,0.1920,0.2311,0.0498
3,Random Forest Classifier,0.7930,0.6781,0.1632,0.6200,0.2488,0.1698,0.2253,0.0568


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=447, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=0)
INFO:logs:compare_models() succesfully completed......................................


单模型训练

In [5]:
# 创建单个模型.
params = {'max_depth': 4, 'min_samples_leaf': 5, 'min_samples_split': 10}
clf = create_model('rf', verbose = True, **params)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8254,0.6585,0.1538,1.0000,0.2667,0.2240,0.3551
1,0.7937,0.8215,0.0769,0.5000,0.1333,0.0829,0.1314
2,0.7937,0.8185,0.1538,0.5000,0.2353,0.1531,0.1889
3,0.7937,0.7323,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7778,0.7085,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.7619,0.7828,0.0000,0.0000,0.0000,-0.0305,-0.0679
6,0.7937,0.6487,0.0714,1.0000,0.1333,0.1069,0.2376
7,0.7778,0.7143,0.0714,0.5000,0.1250,0.0735,0.1210
8,0.7903,0.8006,0.0000,0.0000,0.0000,0.0000,0.0000
9,0.8226,0.6860,0.1538,1.0000,0.2667,0.2232,0.3545


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=447, verbose=0,
                       warm_start=False)
INFO:logs:create_model() succesfully completed......................................


In [6]:
# 模型调参, 使用算法为Stratified Cross Validation.
# TODO: 这里调参时clf中用户设定的参数也发生了改变.
clf_tuned = tune_model(clf, optimize = 'F1', n_iter = 10, fold = 10, round = 4)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8254,0.6346,0.1538,1.0000,0.2667,0.2240,0.3551
1,0.8730,0.8831,0.3846,1.0000,0.5556,0.4980,0.5758
2,0.7937,0.8185,0.3077,0.5000,0.3810,0.2655,0.2767
3,0.7937,0.7954,0.1538,0.5000,0.2353,0.1531,0.1889
4,0.7619,0.6990,0.0714,0.3333,0.1176,0.0426,0.0598
5,0.8254,0.8294,0.3571,0.7143,0.4762,0.3851,0.4185
6,0.7460,0.5802,0.1429,0.3333,0.2000,0.0769,0.0867
7,0.7778,0.6749,0.2143,0.5000,0.3000,0.1923,0.2168
8,0.8065,0.8289,0.2308,0.6000,0.3333,0.2454,0.2840
9,0.7742,0.6931,0.1538,0.4000,0.2222,0.1197,0.1385


INFO:logs:create_model_container: 3
INFO:logs:master_model_container: 3
INFO:logs:display_container: 4
INFO:logs:RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=80, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=447, verbose=0,
                       warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


In [10]:
# 模型评估.
# 提供了Hyperparameters, AUC, Confusion Matrix, Threshold, PR Curve, Error, Class Report
# Feature Selection, Learning Curve, Manifold Learning, etc., 功能比较强.

# plot_model(clf_tuned, plot = 'auc')
evaluate_model(clf_tuned)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
# 模型解释.
# TODO: 偶尔会报错.
interpret_model(clf_tuned)

MLFlow UI

In [ ]:
# TODO: 功能不明.
# !mlflow ui